### 1. ROW_NUMBER vs RANK vs DENSE_RANK (Per Store Top Selling Products)


In [0]:
select s.store_id , p.product_name , round(sum(f.total_amount),1) as total_revenue,

row_number() over(partition by s.store_id order by sum(f.total_amount)desc) as row_n,
rank() over(partition by s.store_id order by sum(f.total_amount)desc) as rank,
dense_rank() over(partition by s.store_id order by sum(f.total_amount)desc) as dense_rank

from learning_pradeep.pipeline.silver_fact_sales f 
left join learning_pradeep.pipeline.silver_dim_stores s on f.store_sk = s.store_sk
left join learning_pradeep.pipeline.silver_dim_products p on f.product_sk = p.product_sk 

group by  p.product_name , s.store_id
order by s.store_id limit 10

### Q2 — TOP N PER CATEGORY (Top 3 Products Per Category)

In [0]:
with ranked as 

(select p.category , p.product_name , sum(f.total_amount) as revenue,

dense_rank() over(partition by p.category order by sum(f.total_amount) desc) as dense_rank

from learning_pradeep.pipeline.silver_fact_sales f 
left join learning_pradeep.pipeline.silver_dim_products p on f.product_sk = p.product_sk 

group by p.category , p.product_name)

select * from ranked where dense_rank <=3

### Q3 — CUMULATIVE SALES (Running Total Over Time)

In [0]:
select date_trunc('month',f.sales_date) as month , sum(f.total_amount) as revenue,

sum(sum(f.total_amount)) over(partition by date_trunc('month',f.sales_date)) as cum_revenue

from learning_pradeep.pipeline.bronze_fact_sales_normalized f 

group by date_trunc('month',f.sales_date)

--order by month

### Q4 — MOVING / ROLLING AVERAGE (Last 3 Months)


In [0]:
SELECT date_trunc('month', f.sales_date) AS month,SUM(f.total_amount) AS monthly_revenue,

AVG(SUM(f.total_amount)) OVER(ORDER BY date_trunc('month', f.sales_date)ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS rolling_3_month_avg

FROM learning_pradeep.pipeline.bronze_fact_sales_normalized f
GROUP BY date_trunc('month', f.sales_date)
ORDER BY month


### Q5 — MONTH ON MONTH (MoM) GROWTH

In [0]:
WITH monthly AS (SELECT date_trunc('month', f.sales_date) AS month,SUM(f.total_amount) AS revenue

FROM learning_pradeep.pipeline.bronze_fact_sales_normalized f 
GROUP BY date_trunc('month',f.sales_date))

select month , revenue ,
lag(revenue,1) over (order by month) pre_month_revenue,

( (revenue - LAG(revenue,1) over(order by month)) 
        / NULLIF(LAG(revenue,1) over(order by month),0)
    ) * 100 as mom_growth_percentage


from monthly
order by month 

### Q7 — CUSTOMER LIFETIME VALUE TREND (Cumulative Spend Per Customer)

In [0]:
SELECT c.customer_id,c.first_name,f.sales_date,f.total_amount,

SUM(f.total_amount) OVER(PARTITION BY c.customer_id ORDER BY f.sales_date) AS cumulative_customer_spend

FROM learning_pradeep.pipeline.bronze_fact_sales_normalized f
LEFT JOIN learning_pradeep.pipeline.silver_dim_customers c ON f.customer_sk = c.customer_sk

ORDER BY c.customer_id, f.sales_date

### Q8 — STORE PERFORMANCE CONTRIBUTION % (Window % Logic)

In [0]:
SELECT s.store_name,SUM(f.total_amount) AS store_revenue,

    SUM(f.total_amount) * 100.0 / 
    SUM(SUM(f.total_amount)) OVER() AS contribution_percentage

FROM learning_pradeep.pipeline.bronze_fact_sales_normalized f
LEFT JOIN learning_pradeep.pipeline.silver_dim_stores s ON f.store_sk = s.store_sk
GROUP BY s.store_name
ORDER BY contribution_percentage DESC 
